# Assignment 5 - Text Analysis
An explanation this assignment could be found in the .pdf explanation document


## Materials to review for this assignment
<h4>From Moodle:</h4> 
<h5><u>Review the notebooks regarding the following python topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Working with strings</b> (tutorial notebook)<br/>
&#x2714; <b>Text Analysis</b> (tutorial notebook)<br/>
&#x2714; <b>Hebrew text analysis tools (tokenizer, wordnet)</b> (moodle example)<br/>
&#x2714; <b>(brief review) All previous notebooks</b><br/>
</div> 
<h5><u>Review the presentations regarding the following topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Text Analysis</b> (lecture presentation)<br/>
&#x2714; <b>(brief review) All other presentations</b><br/>
</div>

## Preceding Step - import modules (packages)
This step is necessary in order to use external modules (packages). <br/>

In [2]:
# --------------------------------------
import pandas as pd
import numpy as np
# --------------------------------------


# --------------------------------------
# ------------- visualizations:
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
# --------------------------------------


# ---------------------------------------
import sklearn
from sklearn import preprocessing, metrics, pipeline, model_selection, feature_extraction 
from sklearn import naive_bayes, linear_model, svm, neural_network, neighbors, tree
from sklearn import decomposition, cluster

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV 
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import mean_squared_error, r2_score, silhouette_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder

from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import Perceptron, SGDClassifier
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
# ---------------------------------------


# ----------------- output and visualizations: 
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.simplefilter("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter("ignore", category=ConvergenceWarning)
# show several prints in one cell. This will allow us to condence every trick in one cell.
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline
pd.pandas.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
# ---------------------------------------

### Text analysis and String manipulation imports:

In [3]:
# --------------------------------------
# --------- Text analysis and Hebrew text analysis imports:
# vectorizers:
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# regular expressions:
import re
# --------------------------------------

### (optional) Hebrew text analysis - WordNet (for Hebrew)
Note: the WordNet is not a must

#### (optional) Only if you didn't install Wordnet (for Hebrew) use:

In [4]:
# word net installation:

# unmark if you want to use and need to install
#!pip install wn
#!python -m wn download omw-he:1.4

In [5]:
# word net import:

# unmark if you want to use:
#import wn

### (optional) Hebrew text analysis - hebrew_tokenizer (Tokenizer for Hebrew)
Note: the hebrew_tokenizer is not a must

#### (optional) Only if you didn't install hebrew_tokenizer use:

In [6]:
# Hebrew tokenizer installation:

# unmark if you want to use and need to install:
#!pip install hebrew_tokenizer

In [7]:
# Hebrew tokenizer import:

# unmark if you want to use:
#import hebrew_tokenizer as ht

### Reading input files
Reading input files for train annotated corpus (raw text data) corpus and for the test corpus

In [8]:
train_filename = 'annotated_corpus_for_train.csv'
test_filename  = 'corpus_for_test.csv'
df_train = pd.read_csv(train_filename, index_col=None, encoding='utf-8')
df_test  = pd.read_csv(test_filename, index_col=None, encoding='utf-8')

In [9]:
df_train.head(8)
df_train.shape

,story,gender
0,"כשחבר הזמין אותי לחול, לא באמת חשבתי שזה יקרה,...",m
1,לפני שהתגייסתי לצבא עשיתי כל מני מיונים ליחידו...,m
2,מאז שהתחילו הלימודים חלומו של כל סטודנט זה הפנ...,f
3,"כשהייתי ילד, מטוסים היה הדבר שהכי ריתק אותי. ב...",m
4,‏הייתי מדריכה בכפר נוער ומתאם הכפר היינו צריכי...,f
5,לפני כ3 חודשים טסתי לרומא למשך שבוע. טסתי במטו...,f
6,אני כבר שנתיים נשוי והשנה אני ואישתי סוף סוף י...,m
7,השנה התחלנו שיפוץ בדירה שלנו בתל אביב. הדירה ה...,f


(753, 2)

In [10]:
df_test.head(3)
df_test.shape
df_train['gender'].value_counts()

,test_example_id,story
0,0,כל קיץ אני והמשפחה נוסעים לארצות הברית לוס אנג...
1,1,"הגעתי לשירות המדינה אחרי שנתיים כפעיל בתנועת ""..."
2,2,אחת האהבות הגדולות שלי אלו הכלבים שלי ושל אישת...


(323, 2)

m    575
f    178
Name: gender, dtype: int64

* The data frame is unbalanced, with significantly more female samples than male samples.
* We will explore how to address this issue further.

### Your implementation:
Write your code solution in the following code-cells

In [11]:
df_train

,story,gender
0,"כשחבר הזמין אותי לחול, לא באמת חשבתי שזה יקרה,...",m
1,לפני שהתגייסתי לצבא עשיתי כל מני מיונים ליחידו...,m
2,מאז שהתחילו הלימודים חלומו של כל סטודנט זה הפנ...,f
3,"כשהייתי ילד, מטוסים היה הדבר שהכי ריתק אותי. ב...",m
4,‏הייתי מדריכה בכפר נוער ומתאם הכפר היינו צריכי...,f
...,...,...
748,אז לפני שנה בדיוק טסתי לאמסטרדם עם שני חברים ט...,m
749,שבוע שעבר העליתי באופן ספונטני רעיון לנסוע עם ...,m
750,לפני חודש עברנו לדירה בבית שמש בעקבות משפחתי ה...,m
751,"החוויה אותה ארצה לשתף התרחשה לפני כמה חודשים, ...",f


In [12]:
# מחיקת תווים מיוחדים 
df_train['story'] = df_train['story'].apply(lambda x: re.sub(r'[^\u0590-\u05FF\s]', '', x))
  
df_train['gender'].replace('f',0,inplace=True)
df_train['gender'].replace('m',1,inplace=True)

df_test['story'] =df_test['story'].apply(lambda x: re.sub(r'[^\u0590-\u05FF\s]', '', x))

df_train

,story,gender
0,כשחבר הזמין אותי לחול לא באמת חשבתי שזה יקרה פ...,1
1,לפני שהתגייסתי לצבא עשיתי כל מני מיונים ליחידו...,1
2,מאז שהתחילו הלימודים חלומו של כל סטודנט זה הפנ...,0
3,כשהייתי ילד מטוסים היה הדבר שהכי ריתק אותי בתו...,1
4,הייתי מדריכה בכפר נוער ומתאם הכפר היינו צריכים...,0
...,...,...
748,אז לפני שנה בדיוק טסתי לאמסטרדם עם שני חברים ט...,1
749,שבוע שעבר העליתי באופן ספונטני רעיון לנסוע עם ...,1
750,לפני חודש עברנו לדירה בבית שמש בעקבות משפחתי ה...,1
751,החוויה אותה ארצה לשתף התרחשה לפני כמה חודשים ז...,0


In [14]:
X=df_train['story']
y=df_train['gender']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df_train['story'],df_train['gender'], test_size=0.2, random_state=42)

In [15]:
# define the vectorizers
vectorizers = [CountVectorizer(), TfidfVectorizer()]

In [16]:
# Define the models and parameters
models = [
    # LinearSVC
    [LinearSVC(),
     {'clf__C': [0.01, 0.1, 1, 10, 100], 'clf__penalty': ['l1', 'l2'], 'clf__class_weight': [None, 'balanced']}],

    # Perceptron
    [Perceptron(),
     {'clf__alpha': [0.0001, 0.001, 0.01, 0.1], 'clf__penalty': ['l2', 'l1', 'elasticnet'], 'clf__class_weight': [None, 'balanced']}],

    # SGDClassifier
    [SGDClassifier(),
     {'clf__alpha': [0.001, 0.05, 0.1], 'clf__loss': ['hinge', 'squared_hinge'], 'clf__class_weight': [None, 'balanced']}],

    # MultinomialNB
    [MultinomialNB(),
     {'clf__alpha': [0.1, 0.5, 1, 10]}],

   # KNeighborsClassifier
    [KNeighborsClassifier(),
     {'clf__n_neighbors': [3, 5, 7, 9], 'clf__weights': ['uniform', 'distance']}],

    # DecisionTreeClassifier
    [DecisionTreeClassifier(),
     {'clf__max_depth': [5, 10, 20, None], 'clf__min_samples_split': [2, 5, 10], 'clf__min_samples_leaf': [1, 2, 4],
      'clf__class_weight': [None, {0: 1, 1: 10}]}]
]



# Use GridSearchCV to find the best parameters for each classifier


In [19]:
from sklearn.model_selection import StratifiedKFold

In [20]:
def evaluate_model(model, parameters, X, y, vectorizer):
    skf = StratifiedKFold(n_splits=10)
    
    param_grid = {
        'vect__max_features': [10_000, 30_000, 40_000],
        'vect__ngram_range': [(1, 1), (1, 2), (1, 3)],
        'vect__min_df': [1, 3],
        'vect__max_df': [0.7, 0.9],
        **parameters
    }

    pipeline = Pipeline([('vect', vectorizer), ('clf', model)])
    grid_search = GridSearchCV(pipeline, param_grid, cv=skf, scoring='f1_macro')
    grid_search.fit(X, y)

    results_df = pd.DataFrame(grid_search.cv_results_)

    print("Model Type: ", type(model).__name__)
    print("Vectorizer Type: ", type(vect).__name__)


    print()
    print("Best hyperparameters: %s" % grid_search.best_params_)
    print("Cross-validation score: %0.2f" % grid_search.best_score_)
    print("********************************************************")


    best_model = grid_search.best_estimator_

    return best_model

# LinearSVC

In [21]:
clf=models[0][0]
param=models[0][1]
vect=vectorizers[0]
best_model_1=evaluate_model(clf,param,X,y,vect)

Model Type:  LinearSVC
Vectorizer Type:  CountVectorizer

Best hyperparameters: {'clf__C': 0.01, 'clf__class_weight': 'balanced', 'clf__penalty': 'l2', 'vect__max_df': 0.9, 'vect__max_features': 10000, 'vect__min_df': 3, 'vect__ngram_range': (1, 1)}
Cross-validation score: 0.70
********************************************************


In [22]:
vect=vectorizers[1]
best_model_2=evaluate_model(clf,param,X,y,vect)

Model Type:  LinearSVC
Vectorizer Type:  TfidfVectorizer

Best hyperparameters: {'clf__C': 0.01, 'clf__class_weight': 'balanced', 'clf__penalty': 'l2', 'vect__max_df': 0.9, 'vect__max_features': 10000, 'vect__min_df': 3, 'vect__ngram_range': (1, 2)}
Cross-validation score: 0.70
********************************************************


# Perceptron 

In [23]:
clf=models[1][0]
vect=vectorizers[0]
best_model_3=evaluate_model(clf,models[1][1],X,y,vect)

Model Type:  Perceptron
Vectorizer Type:  CountVectorizer

Best hyperparameters: {'clf__alpha': 0.0001, 'clf__class_weight': 'balanced', 'clf__penalty': 'l1', 'vect__max_df': 0.9, 'vect__max_features': 10000, 'vect__min_df': 3, 'vect__ngram_range': (1, 3)}
Cross-validation score: 0.71
********************************************************


In [24]:
vect=vectorizers[1]
best_model_4=evaluate_model(clf,models[1][1],X,y,vect)

Model Type:  Perceptron
Vectorizer Type:  TfidfVectorizer

Best hyperparameters: {'clf__alpha': 0.0001, 'clf__class_weight': 'balanced', 'clf__penalty': 'elasticnet', 'vect__max_df': 0.9, 'vect__max_features': 30000, 'vect__min_df': 3, 'vect__ngram_range': (1, 2)}
Cross-validation score: 0.71
********************************************************


# SGDClassifier

In [25]:
clf=models[2][0]
vect=vectorizers[0]
best_model_5=evaluate_model(clf,models[2][1],X,y,vect)

vect=vectorizers[1]
best_model_6=evaluate_model(clf,models[2][1],X,y,vect)

Model Type:  SGDClassifier
Vectorizer Type:  CountVectorizer

Best hyperparameters: {'clf__alpha': 0.05, 'clf__class_weight': 'balanced', 'clf__loss': 'hinge', 'vect__max_df': 0.9, 'vect__max_features': 30000, 'vect__min_df': 3, 'vect__ngram_range': (1, 3)}
Cross-validation score: 0.72
********************************************************
Model Type:  SGDClassifier
Vectorizer Type:  TfidfVectorizer

Best hyperparameters: {'clf__alpha': 0.05, 'clf__class_weight': 'balanced', 'clf__loss': 'squared_hinge', 'vect__max_df': 0.7, 'vect__max_features': 40000, 'vect__min_df': 3, 'vect__ngram_range': (1, 1)}
Cross-validation score: 0.71
********************************************************


# MultinomialNB

In [26]:
clf=models[3][0]
vect=vectorizers[0]
best_model_7=evaluate_model(clf,models[3][1],X,y,vect)

vect=vectorizers[1]
best_model_8=evaluate_model(clf,models[3][1],X,y,vect)

Model Type:  MultinomialNB
Vectorizer Type:  CountVectorizer

Best hyperparameters: {'clf__alpha': 0.5, 'vect__max_df': 0.9, 'vect__max_features': 10000, 'vect__min_df': 1, 'vect__ngram_range': (1, 2)}
Cross-validation score: 0.70
********************************************************
Model Type:  MultinomialNB
Vectorizer Type:  TfidfVectorizer

Best hyperparameters: {'clf__alpha': 0.1, 'vect__max_df': 0.7, 'vect__max_features': 10000, 'vect__min_df': 3, 'vect__ngram_range': (1, 3)}
Cross-validation score: 0.54
********************************************************


# KNeighborsClassifier

In [27]:
clf=models[4][0]
vect=vectorizers[0]
best_model_9=evaluate_model(clf,models[4][1],X,y,vect)

vect=vectorizers[1]
best_model_10=evaluate_model(clf,models[4][1],X,y,vect)

Model Type:  KNeighborsClassifier
Vectorizer Type:  CountVectorizer

Best hyperparameters: {'clf__n_neighbors': 5, 'clf__weights': 'distance', 'vect__max_df': 0.9, 'vect__max_features': 30000, 'vect__min_df': 1, 'vect__ngram_range': (1, 3)}
Cross-validation score: 0.55
********************************************************
Model Type:  KNeighborsClassifier
Vectorizer Type:  TfidfVectorizer

Best hyperparameters: {'clf__n_neighbors': 5, 'clf__weights': 'distance', 'vect__max_df': 0.9, 'vect__max_features': 30000, 'vect__min_df': 1, 'vect__ngram_range': (1, 3)}
Cross-validation score: 0.63
********************************************************


# DecisionTreeClassifier

In [28]:
clf=models[5][0]
vect=vectorizers[0]
best_model_11=evaluate_model(clf,models[5][1],X,y,vect)

vect=vectorizers[1]
best_model_12=evaluate_model(clf,models[5][1],X,y,vect)

Model Type:  DecisionTreeClassifier
Vectorizer Type:  CountVectorizer

Best hyperparameters: {'clf__class_weight': {0: 1, 1: 10}, 'clf__max_depth': None, 'clf__min_samples_leaf': 2, 'clf__min_samples_split': 10, 'vect__max_df': 0.9, 'vect__max_features': 30000, 'vect__min_df': 3, 'vect__ngram_range': (1, 2)}
Cross-validation score: 0.73
********************************************************
Model Type:  DecisionTreeClassifier
Vectorizer Type:  TfidfVectorizer

Best hyperparameters: {'clf__class_weight': {0: 1, 1: 10}, 'clf__max_depth': None, 'clf__min_samples_leaf': 1, 'clf__min_samples_split': 5, 'vect__max_df': 0.7, 'vect__max_features': 40000, 'vect__min_df': 1, 'vect__ngram_range': (1, 3)}
Cross-validation score: 0.71
********************************************************


# LogisticRegression

In [29]:
from sklearn.linear_model import LogisticRegression
clf=LogisticRegression()
param={'clf__C': [0.01, 0.1, 1, 10],
       'clf__class_weight': [None,'balanced']}

vect=vectorizers[0]
best_model_13=evaluate_model(clf,param,X,y,vect)

vect=vectorizers[1]
best_model_14=evaluate_model(clf,param,X,y,vect)

Model Type:  LogisticRegression
Vectorizer Type:  CountVectorizer

Best hyperparameters: {'clf__C': 0.01, 'clf__class_weight': 'balanced', 'vect__max_df': 0.9, 'vect__max_features': 10000, 'vect__min_df': 3, 'vect__ngram_range': (1, 3)}
Cross-validation score: 0.70
********************************************************
Model Type:  LogisticRegression
Vectorizer Type:  TfidfVectorizer

Best hyperparameters: {'clf__C': 0.1, 'clf__class_weight': 'balanced', 'vect__max_df': 0.9, 'vect__max_features': 10000, 'vect__min_df': 3, 'vect__ngram_range': (1, 1)}
Cross-validation score: 0.70
********************************************************


In [100]:
best_model=best_model_11
best_model
skf=StratifiedKFold(n_splits=10)

f1_scores = cross_val_score(best_model, X, y, cv=skf, scoring='f1_macro')

mean_f1_score = np.mean(f1_scores)

std_dev = np.std(f1_scores)
f1_scores

print("Mean F1 Score:", round(mean_f1_score, 2))
print("Standard Deviation:", round(std_dev, 2))

Pipeline(steps=[('vect',
                 CountVectorizer(max_df=0.9, max_features=30000, min_df=3,
                                 ngram_range=(1, 2))),
                ('clf',
                 DecisionTreeClassifier(class_weight={0: 1, 1: 10},
                                        min_samples_leaf=2,
                                        min_samples_split=10))])

array([0.72291667, 0.78711485, 0.68449328, 0.73570073, 0.66124661,
       0.80137712, 0.76165254, 0.72192797, 0.70760234, 0.70833333])

Mean F1 Score: 0.73
Standard Deviation: 0.04


# result

# best model
## DecisionTreeClassifier   
### The model was built using the following parameters:

### CountVectorizer with the following settings:

* max_df: 0.9
* max_features: 30000
* min_df: 3
* ngram_range: (1, 2)

### DecisionTreeClassifier  with the following settings:

* class_weight={0: 1, 1: 10}
* min_samples_leaf=2
* min_samples_split=10

#### These parameter choices allow for an improved representation of the text by focusing on important words and assigning higher value to words that appear in the current document and in other documents in the corpus.

## StratifiedKFold

#### StratifiedKFold is a useful method for performing cross-validation on datasets with imbalanced class distributions. It ensures that each fold contains a proportional representation of samples from each class, which helps in obtaining reliable and unbiased performance estimates for the model. By using StratifiedKFold, you can assess the model's stability and generalization capabilities more accurately.

#### The F1 scores obtained from the cross-validation are as follows:
* [0.72291667, 0.78711485, 0.68449328, 0.73570073, 0.66124661, 0.80137712, 0.76165254, 0.72192797, 0.70760234, 0.70833333]

#### The mean F1 score is 0.73, indicating the average performance of the model across the different folds. The standard deviation is 0.04, which represents the variability or spread of the F1 scores.

### Overall, this model demonstrates stable performance with a relatively high mean F1 score and a moderate standard deviation.
##### Further analysis and iterations are recommended to fine-tune the model and address the class imbalance issue in order to achieve better results.

# predict 

In [105]:
df_test_tokenized = df_test.copy()
y_predict_test=best_model.predict(df_test_tokenized['story'])
df_test['gender']=y_predict_test
df_test['gender'].replace(0,'f',inplace=True)
df_test['gender'].replace(1,'m',inplace=True)

In [112]:
df_predicted=df_test.copy()
df_predicted.loc[:,['story','gender']]

,story,gender
0,כל קיץ אני והמשפחה נוסעים לארצות הברית לוס אנג...,m
1,הגעתי לשירות המדינה אחרי שנתיים כפעיל בתנועת י...,m
2,אחת האהבות הגדולות שלי אלו הכלבים שלי ושל אישת...,m
3,רגע הגיוס לצבא היה הרגע הכי משמעותי עבורי אני ...,m
4,אני הגעתי לברזיל ישר מקולומביה וגם אני עשיתי ע...,m
...,...,...
318,בשנה האחרונה הרגשתי די תקוע בעבודה השגרה הפכה ...,m
319,אני ואילן חברים טובים מזה שנה תמיד חלמנו לפתח...,m
320,מידי יום שישי אני נוהג לנסוע בתחבורה ציבורית ס...,f
321,לפני מספר חודשים בשיא התחלואה של הגל השני עמית...,m


### Save output to csv (optional)
After you're done save your output to the 'classification_results.csv' csv file.<br/>
We assume that the dataframe with your results contain the following columns:
* column 1 (left column): 'test_example_id'  - the same id associated to each of the test stories to be predicted.
* column 2 (right column): 'predicted_category' - the predicted gender value for each of the associated story. 

Assuming your predicted values are in the `df_predicted` dataframe, you should save you're results as following:

In [113]:
df_predicted.to_csv('classification_results.csv',index=False)